# distilbert
https://huggingface.co/docs/transformers/model_doc/distilbert


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel


In [ ]:

# device = "cuda" # the device to load the model onto
device = "cpu" # the device to load the model onto

tokenizer = AutoTokenizer.from_pretrained('distilbert/distilbert-base-uncased')
model = AutoModel.from_pretrained("distilbert/distilbert-base-uncased", torch_dtype=torch.float16, attn_implementation="flash_attention_2")


In [ ]:

text = "Replace me by any text you'd like."

encoded_input = tokenizer(text, return_tensors='pt').to(device)
model.to(device)

output = model(**encoded_input)

# google Gemma
https://huggingface.co/google/gemma-2-9b

In [ ]:
# pip install accelerate
# pip install transformers[torch]
# pip install ipywidgets
# もしくは
# pip install -r requiremets.txt

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [3]:

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-9b")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-9b",
    device_map="auto",
    torch_dtype=torch.bfloat16
)


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/google/gemma-2-9b.
401 Client Error. (Request ID: Root=1-66873ee3-45a5c76834cae61a35b65299;8994a406-1c2a-4867-9a54-7e105b88f969)

Cannot access gated repo for url https://huggingface.co/google/gemma-2-9b/resolve/main/config.json.
Access to model google/gemma-2-9b is restricted. You must be authenticated to access it.

In [ ]:
input_text = "Write me a poem about Machine Learning."
# input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")
input_ids = tokenizer(input_text, return_tensors="pt")
input_ids

In [ ]:
outputs = model.generate(**input_ids)
outputs

In [ ]:
print(tokenizer.decode(outputs[0]))


# elyza
https://huggingface.co/elyza/ELYZA-japanese-Llama-2-7b

In [7]:
import torch
import os
from transformers import AutoModelForCausalLM, AutoTokenizer


In [9]:
# win only? to disapear warnings
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "true"

In [10]:

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = "あなたは誠実で優秀な日本人のアシスタントです。"
text = "クマが海辺に行ってアザラシと友達になり、最終的には家に帰るというプロットの短編小説を書いてください。"


In [11]:

model_name = "elyza/ELYZA-japanese-Llama-2-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto")

# C:/Users/*/.cache/huggingface/にモデルが保存される

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

: 

windowsで実行すると下記ワーニングが出た
Windowsを開発者モードにすることとが以下のリンク先に記載。また、このエラーを表示しないようにするには環境変数を設定することが記載されている
参考URL
https://huggingface.co/docs/huggingface_hub/v0.23.4/guides/manage-cache#limitations


c:\Users\*\Desktop\study_huggingface\hug_env\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\*\.cache\huggingface\hub\models--elyza--ELYZA-japanese-Llama-2-7b-instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Downloading shards:   0%|          | 0/2 [00:00<?, ?it/s]


c:\Users\A067214\Desktop\study_huggingface\hug_env\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\A067214\.cache\huggingface\hub\models--elyza--ELYZA-japanese-Llama-2-7b-instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Downloading shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:

if torch.cuda.is_available():
    model = model.to("cuda")
else:
    model = model.to('cpu')

prompt = "{bos_token}{b_inst} {system}{prompt} {e_inst} ".format(
    bos_token=tokenizer.bos_token,
    b_inst=B_INST,
    system=f"{B_SYS}{DEFAULT_SYSTEM_PROMPT}{E_SYS}",
    prompt=text,
    e_inst=E_INST,
)



In [ ]:
with torch.no_grad():
    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=256,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )


In [ ]:
output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1) :], skip_special_tokens=True)
print(output)
"""
承知しました。以下にクマが海辺に行ってアザラシと友達になり、最終的には家に帰るというプロットの短編小説を記述します。

クマは山の中でゆっくりと眠っていた。
その眠りに落ちたクマは、夢の中で海辺を歩いていた。
そこにはアザラシがいた。
クマはアザラシに話しかける。

「おはよう」とクマが言うと、アザラシは驚いたように顔を上げた。
「あ、こんにちは」アザラシは答えた。
クマはアザラシと友達になりたいと思う。

「私はクマと申します。」クマは...
"""


# Phi-3
https://huggingface.co/microsoft/Phi-3-mini-4k-instruct

In [8]:
import torch 
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline 


In [9]:

torch.random.manual_seed(0) 
model = AutoModelForCausalLM.from_pretrained( 
    "microsoft/Phi-3-mini-4k-instruct",  
    device_map="cpu",  
    torch_dtype="auto",  
    trust_remote_code=True,
    attn_implementation='eager',
) 


`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct") 


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:

messages = [ 
    {"role": "system", "content": "You are a helpful AI assistant."}, 
    {"role": "user", "content": "Can you provide ways to eat combinations of bananas and dragonfruits?"}, 
    {"role": "assistant", "content": "Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey."}, 
    {"role": "user", "content": "What about solving an 2x + 3 = 7 equation?"}, 
] 


In [12]:

pipe = pipeline( 
    "text-generation", 
    model=model, 
    tokenizer=tokenizer, 
) 


In [13]:

generation_args = { 
    "max_new_tokens": 500, 
    "return_full_text": False, 
    "temperature": 0.0, 
    "do_sample": False, 
} 


In [14]:

output = pipe(messages, **generation_args) 
print(output[0]['generated_text'])


c:\Users\A067214\Desktop\study_huggingface\hug_env\lib\site-packages\transformers\generation\configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


phi-1
https://huggingface.co/microsoft/phi-1

In [16]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer


In [17]:

torch.set_default_device("cpu")

model = AutoModelForCausalLM.from_pretrained("microsoft/phi-1", torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1")


model.safetensors:   0%|          | 10.5M/2.84G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:

inputs = tokenizer('''def print_prime(n):
   """
   Print all primes between 1 and n
   """''', return_tensors="pt", return_attention_mask=False)
inputs

In [ ]:

outputs = model.generate(**inputs, max_length=200)
text = tokenizer.batch_decode(outputs)[0]
print(text)


# Phi-3-mini-4k-instruct-gguf
https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf